In [1]:
import numpy as np
import sys
import os 
from functools import partial
import scipy
from scipy.optimize import fmin
DATA_DIR = r"C:\Users\julia\VSCode\MSiD\data"

Exercise 1
W metodzie stałokrokowej nie mamy gwarancji osiągnięcia minimum, gdyż przy stałym kroku możemy nawet przeskoczyć minimum lokalne 

In [2]:
def steepest_descent(func, gradient, learning_rate, start_point, stop_condition, write_to=None):
    current_point = start_point
    i = 1
    while True:
        print(f"ITERATION {i}", file=write_to)
        gradient_value = gradient(current_point)
        print(f"\tgradient_value = {gradient_value}", file=write_to)
        step_size = learning_rate * gradient_value
        print(f"\tstep_size = {step_size}", file=write_to)
        next_point = current_point - step_size 
        print(f"\tnext_point = {next_point}", file=write_to)
        print(f"\tf({next_point}) = {func(next_point)}", file=write_to)

        if stop_condition(current_point, next_point, write_to=write_to):
            print(f"Reached stop condition at {next_point}, value = {func(next_point)}", file=write_to)
            break 
        i += 1
        current_point = next_point
    

    return current_point


def func(X):
    return np.sum(X ** 2)

def gradient(X):
    return X * 2

def L1_norm(X):
    return np.sum(np.abs(X))

def stop(current_point, next_point, error, write_to=None):
    computed_L1 = L1_norm(current_point - next_point)
    print(f"L1 = {computed_L1}", file=write_to) 
    return computed_L1 < error 


In [3]:
h_i = 0.3
x_start = np.array([4, 4])
with open(os.path.join(DATA_DIR, "ex1.txt"), mode='w') as f:
    result = steepest_descent(func, gradient, h_i, x_start, partial(stop, error=0.01), write_to=f)
    print(f"f({result}) = {func(result)}")

f([0.0065536 0.0065536]) = 8.589934592000008e-05


In [4]:
v1 = np.array([3, 4])
v2 = np.array([1, 1])

# Subtraction is element-wise
v1 - v2

array([2, 3])

In [5]:
scipy_min = fmin(func, x0=x_start)
func(scipy_min)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 44
         Function evaluations: 80


9.503461544006107e-10

Exercise 2
Minimum tej funkcji znajduje się w punkcie (1, 1)

In [6]:
def func(X):
    return 2.5 * (X[0] ** 2 - X[1]) ** 2 + (1 - X[0]) ** 2

def gradient(X):
    return np.array([10 * X[0] ** 3 - 10 * X[0] * X[1] + 2 * X[0] - 2, 
                     -5 * (X[0] ** 2 - X[1])])

h_i = 0.1
x_start = np.array([-0.5, 1])
with open(os.path.join(DATA_DIR, "ex2.txt"), mode='w') as f:
    result = steepest_descent(func, gradient, h_i, x_start, partial(stop, error=0.1), write_to=f)
    print(f"f({result}) = {func(result)}")

f([0.54009587 0.18933999]) = 0.237707553462972


In [7]:
h_i = 0.1
# Iteration 37
with open(os.path.join(DATA_DIR, "ex2_a.txt"), mode='w') as f:
    result = steepest_descent(func, gradient, h_i, x_start, partial(stop, error=0.01), write_to=f)
    print(f"f({result}) = {func(result)}")

h_i = 0.01
with open(os.path.join(DATA_DIR, "ex2_b.txt"), mode='w') as f:
    result = steepest_descent(func, gradient, h_i, x_start, partial(stop, error=0.001), write_to=f)
    print(f"f({result}) = {func(result)}")

f([0.9219545  0.83663145]) = 0.0065379022322644325
f([0.92197522 0.83668903]) = 0.00653337401708192


In [8]:
func(np.array([1, 2]))

2.5

In [9]:
scipy_min = fmin(func, x0=x_start)
func(scipy_min)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 58
         Function evaluations: 109


2.3379829250198353e-10

W porównaniu z metodą zmiennokrokową, w metodzie stałokrokowej nie wykonujemy dodatkowych obliczeń, więc możemy więcej iteracji zrobić

Exercise 3

In [10]:
def var_steepest_descent(func, gradient, learning_rate, start_point, stop_condition, write_to=None):
    current_point = start_point
    i = 1
    while True:
        print(f"ITERATION {i}", file=write_to)
        gradient_value = gradient(current_point)
        print(f"\tgradient_value = {gradient_value}", file=write_to)
        step_size = learning_rate(current_point, gradient_value, write_to) * gradient_value
        print(f"\tstep_size = {step_size}", file=write_to)
        next_point = current_point - step_size 
        print(f"\tnext_point = {next_point}", file=write_to)
        print(f"\tf({next_point}) = {func(next_point)}", file=write_to)

        if stop_condition(current_point, next_point, write_to=write_to):
            print(f"Reached stop condition at {next_point}, value = {func(next_point):.2f}", file=write_to)
            break 
        i += 1
        current_point = next_point
    

    return current_point


def update_learning_rate(X, gradient, write_to=None):
    
    def metric(alpha):
        # This is original function
        return np.sum((X - alpha * gradient) ** 2)
    
    min_alpha = fmin(metric, x0=0.01, disp=False)
    print(f"\talpha = {min_alpha[0]:.2f}", file=write_to)
    return min_alpha[0]

W metodzie zmiennokrokowej wykonujemy optymalizację kierunkową współczynnika kroku. Tutaj wybieramy takie $\alpha$, by minimalizowało funkcję wejściową

In [11]:
update_learning_rate(np.array([4, 4]), np.array([8, 8]))

	alpha = 0.50


0.5000000000000007

Tutaj potrzebowaliśmy jednego kroku, by znaleźć minimum. 

In [12]:
def func(X):
    return np.sum(X ** 2)

def gradient(X):
    return X * 2

x_start = np.array([4, 4])
with open(os.path.join(DATA_DIR, "ex3.txt"), mode='w') as f:
    result = var_steepest_descent(func, gradient, update_learning_rate, x_start, partial(stop, error=0.01), write_to=f)
    print(f"f({result}) = {func(result)}")

f([-5.32907052e-15 -5.32907052e-15]) = 5.679798517591285e-29


Exercise 4

In [13]:
def var_steepest_descent_2(func, gradient, learning_rate, start_point, stop_condition, write_to=None):
    current_point = start_point
    i = 1
    while True:
        print(f"ITERATION {i}", file=write_to)
        gradient_value = gradient(current_point)
        print(f"\tgradient_value = {gradient_value}", file=write_to)
        step_size = learning_rate(i, write_to) * gradient_value
        print(f"\tstep_size = {step_size}", file=write_to)
        next_point = current_point - step_size 
        print(f"\tnext_point = {next_point}", file=write_to)
        print(f"\tf({next_point}) = {func(next_point)}", file=write_to)

        if stop_condition(current_point, next_point, write_to=write_to):
            print(f"Reached stop condition at {next_point}, value = {func(next_point):.2f}", file=write_to)
            break 
        i += 1
        current_point = next_point
    

    return current_point

In [14]:
def lr1(i, write_to=None):
    next_step = 0.0007 * i 
    print(f"\tnext step = {next_step:.2f}", file=write_to)
    return next_step

def lr2(i, write_to=None):
    next_step = 0.000701 * i 
    print(f"\tnext step = {next_step:.2f}", file=write_to)
    return next_step

def lr3(i, write_to=None):
    next_step = 0.03 * np.sqrt(i) 
    print(f"\tnext step = {next_step:.2f}", file=write_to)
    return next_step

def lr4(i, write_to=None):
    next_step = 0.03* np.sqrt(np.sqrt(i))
    print(f"\tnext step = {next_step:.2f}", file=write_to)
    return next_step


In [15]:
methods = [lr1, lr2, lr3, lr4]
names = ['a', 'b', 'c', 'd']

def func(X):
    return 2.5 * (X[0] ** 2 - X[1]) ** 2 + (1 - X[0]) ** 2

def gradient(X):
    return np.array([10 * X[0] ** 3 - 10 * X[0] * X[1] + 2 * X[0] - 2, 
                     -5 * (X[0] ** 2 - X[1])])


x_start = np.array([-0.5, 1])

for method, name in zip(methods, names):
    with open(os.path.join(DATA_DIR, f"ex4{name}.txt"), mode='w') as f:
        result = var_steepest_descent_2(func, gradient, method, x_start, partial(stop, error=0.001), write_to=f)
        print(f"[{name}]: f({result}) = {func(result)}")

[a]: f([0.9931332  0.98519829]) = 5.026245143622128e-05
[b]: f([0.9931852  0.98527724]) = 4.9688200256330327e-05


C:\Users\julia\AppData\Local\Temp\ipykernel_8604\246558139.py:5: RuntimeWarning: overflow encountered in double_scalars
  return 2.5 * (X[0] ** 2 - X[1]) ** 2 + (1 - X[0]) ** 2
C:\Users\julia\AppData\Local\Temp\ipykernel_8604\246558139.py:8: RuntimeWarning: overflow encountered in double_scalars
  return np.array([10 * X[0] ** 3 - 10 * X[0] * X[1] + 2 * X[0] - 2,
C:\Users\julia\AppData\Local\Temp\ipykernel_8604\246558139.py:9: RuntimeWarning: overflow encountered in double_scalars
  -5 * (X[0] ** 2 - X[1])])
C:\Users\julia\AppData\Local\Temp\ipykernel_8604\246558139.py:5: RuntimeWarning: invalid value encountered in double_scalars
  return 2.5 * (X[0] ** 2 - X[1]) ** 2 + (1 - X[0]) ** 2
C:\Users\julia\AppData\Local\Temp\ipykernel_8604\246558139.py:8: RuntimeWarning: invalid value encountered in double_scalars
  return np.array([10 * X[0] ** 3 - 10 * X[0] * X[1] + 2 * X[0] - 2,
C:\Users\julia\AppData\Local\Temp\ipykernel_8604\246558139.py:9: RuntimeWarning: invalid value encountered in 